# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770457888715                   -0.52    9.0    188ms
  2   -2.771693973307       -2.91       -1.32    1.0   95.3ms
  3   -2.771714369151       -4.69       -2.45    1.0   94.9ms
  4   -2.771714644507       -6.56       -3.15    1.0    101ms
  5   -2.771714714944       -7.15       -4.23    2.0    118ms
  6   -2.771714715151       -9.68       -4.57    1.0    103ms
  7   -2.771714715248      -10.01       -5.82    1.0    443ms
  8   -2.771714715250      -11.77       -6.39    2.0    138ms
  9   -2.771714715250      -13.36       -7.25    1.0    111ms
 10   -2.771714715250   +  -13.73       -8.12    2.0    132ms


-0.000134573569711641

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770553358091                   -0.53    9.0    185ms
  2   -2.771772632480       -2.91       -1.32    1.0    101ms
  3   -2.771801437649       -4.54       -2.54    1.0   96.6ms
  4   -2.771802018256       -6.24       -3.49    1.0   97.6ms
  5   -2.771802072954       -7.26       -3.94    2.0    118ms
  6   -2.771802074421       -8.83       -5.16    1.0    142ms
  7   -2.771802074473      -10.29       -5.29    2.0    420ms
  8   -2.771802074476      -11.52       -6.39    1.0    107ms
  9   -2.771802074476      -13.09       -6.76    2.0    144ms
 10   -2.771802074476   +  -14.57       -7.40    1.0    102ms
 11   -2.771802074476   +  -14.24       -8.01    2.0    120ms


0.017612219004175415

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.000134573569711641
Displaced dipole:  0.017612219004175415
Polarizability :   1.7746792573887056


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, -δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0   5.72s  Non-interacting
   1        0       1       -0.60     10.0   13.0s  
   2        0       2       -2.42      8.0   556ms  
   3        0       3       -3.54      5.0   103ms  
   4        0       4       -5.33      4.0  96.9ms  
   5        0       5       -7.20      1.0  84.0ms  
   6        0       6      -10.30      1.0  83.1ms  
   7        1       1       -7.35     13.0   723ms  Restart
   8        1       2       -8.89      1.0  95.1ms  
                                      13.0   776ms  Final orbitals


(δψ = Matrix{ComplexF64}[[-0.0021621635018363255 - 0.002487564075539402im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.23521185763223143 + 0.20100143191728684im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; -0.031102109481848363 + 0.026814280325459048im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.06088395111253302 - 0.0533304445797549im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.6206936919693087e-7 -2.5020146570336327e-7 … -4.880640512021055e-8 -2.5020144390028993e-7; -6.293614740807892e-7 -4.793034883287647e-7 … -1.2077835265786553e-7 -4.793034684748374e-7; … ; 1.362625129679453e-7 1.1356220363437618e-7 … 4.711983102455197e-8 1.1356221174079263e-7; 1.3305465733165291e-7 1.3632631066006587e-7 … 5.2006819662933965e-8 1.3632632661491542e-7;;; -2.502015141709145e-7 -1.7404908429463868e-7 … -3.5564928766257435e-8 -1.740490674541649e-7; -4.793035190229582e-7 -3.6797309938976644e-7 … -1.1202886622597404e-7 -3.679730813448645e-7; … ; 1.1356218575076811e-7 1.1211086414383064e-7 … 1.0980673232413136e-7 1

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.925712512900697
Interacting polarizability:     1.7736548443898752


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 1.09e-09
[ Info: GMRES linsolve in iteration 2; step 1: normres = 7.14e-11
┌ Info: GMRES linsolve converged at iteration 2, step 2:
│ * norm of residual = 3.66e-12
└ * number of operations = 11
Non-interacting polarizability: 1.9257125129005914
Interacting polarizability:     1.7736548392481233


We obtain the identical result to above.